# Build Your First Neural Network with PyTorch
Copyright 2022, LEAKY.AI LLC

This exercise is intended for anyone curious about building A.I. projects.  In order to complete this tutorial, follow along at https://www.leaky.ai. 

In this tutorial, we will build a neural network from scratch.  You will be introduced to PyTorch, a deep learning library managed by Meta's AI group and the power of neural networks.  We will be working on a synthetic dataset that catalogs the daily number of lemons sold at a lemon stand.  After training, the neural network will be ablet to predict the number of lemons that are likely going to be sold on a given day.  Lets get started!

### Option 1 - If using Google Colab
In order to complete this assignment:
1.	Copy the following link: https://github.com/LeakyAI/FirstNeuralNet
2.	Head over to Colab: https://colab.research.google.com/
3.	Click on GitHub and paste in the repo link above
4.	Click the magnify icon on the right side of the link you pasted above
5.	Click on the <b>FirstNeuralNetwork - Start Here.ipynb</b> notebook to get started

### Option 2 - If you are running the notebook directly on your own laptop/desktop
If you are using your own laptop or desktop and running the notebook locally, we recommend you complete the tutorial on how to configure your PC for A.I. at www.leaky.ai first and then attempt this tutorial.

Next, replace the <b>[TBD]</b> parts below with your code and execute all the cells.  Most cells will have the correct solution shown below the cell for you to check your results.

Good luck!

# Step 1 - Setup our Environment
Lets start by importing the software libraries we will need to build our neural network.  We will import PyTorch and check the version of PyTorch.  You will usually want to run the latest version.  Check the link below to verify you are running the latest version of PyTorch:
    
    https://pytorch.org/

In [1]:
# Import PyTorch libraries
import torch
from torch import nn 

# Verify PyTorch libraries have been loaded
torch.__version__

'1.9.0'

### Check Our Processing Capability (CPU vs. GPU)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('A {} device was found for processing'.format(device))

A cuda device was found for processing


### Download the Dataset
When training a neural network from scratch you will usually need a lot of data.  We will start by loading all the lemonade stand data which is a simply synthetic dataset.  It includes various fields like whether it was sunny, it was a weekend etc. on the day the sales took place.  The last field contains the number of lemonades actually sold.  This is the field we will want to train our neural network to predict.

In [3]:
# Use Pandas to do our dataprocessing on the dataset
import pandas as pd
url = 'https://raw.githubusercontent.com/LeakyAI/FirstNeuralNet/main/lemons.csv'
df = pd.read_csv(url)
df.head(5)

,Weekend,Afternoon,Sunny,Warm,Sign,CuteDog,Ballons,Discount,FreshlyPicked,Ice,CleanHouse,DogWalked,NumberLemonsSold
0,1,1,1,0,1,0,0,0,0,1,0,1,10
1,1,0,1,0,1,0,0,0,0,0,1,0,7
2,0,0,1,0,0,1,1,0,0,0,0,0,0
3,0,0,1,1,0,1,0,0,1,1,1,1,0
4,1,1,1,0,0,1,0,0,1,1,0,0,14


# Step 2 - Prep our Data for the Neural Network

The data has been collected in a table with the following columns:  'Weekend', 'Afternoon', 'Sunny', 'Warm', 'Sign', 'CuteDog', 'Ballons', 'Discount', 'FreshlyPicked', 'Ice', 'CleanHouse', 'DogWalked', 'NumberLemonsSold'.  The number we want our neural network to predict is the field called 'NumberLemonsSold'!  In order to train our network, we will need to seperate out our "input" (x) to the neural network and the "output" (y) or value we want it to learn.  Below we take out the last column from the data and place it into "y".

In [4]:
# Read only the columns of the csv file
df = pd.read_csv(url, nrows=1) 
columns = df.columns.tolist() 
print ("Columns: ", columns)

# Remove the last column from the list
cols_to_use = columns[:len(columns)-1] # drop the last one

# Now build our input (x) and output (y) to the neural network
xDF = pd.read_csv(url, usecols=cols_to_use)
yDF = pd.read_csv(url, usecols=['NumberLemonsSold'])

# Finally convert our data to PyTorch tensors
x = torch.tensor(xDF.values,dtype=torch.float)
y = torch.tensor(yDF.values,dtype=torch.float)

# Move our data to either CPU or GPU depending on what we discovered above
x = x.to(device)
y = y.to(device)

Columns:  ['Weekend', 'Afternoon', 'Sunny', 'Warm', 'Sign', 'CuteDog', 'Ballons', 'Discount', 'FreshlyPicked', 'Ice', 'CleanHouse', 'DogWalked', 'NumberLemonsSold']


# Step 3 - Build our Neural Network
Below we build a simply neural network that will take in the inputs above (12) and produce a single value as an output.  This network has a single hidden layer of 30 units.

In [5]:
model = nn.Sequential(
            nn.Linear(12,30),
            nn.ReLU(),
            nn.Linear(30,1)
        )

# Move it to either the CPU or GPU depending on what we have available
model.to(device)

Sequential(
  (0): Linear(in_features=12, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=1, bias=True)
)

# Step 4 - Train our Neural Network

Here we will simply train our neural network on the dataset.  We will provide it with an input and and output.  The training loop will then adjust the weights within the neural network to make it more accuarate as we go through the training process.

In [6]:
import torch.optim as optim

# Meausure our neural network by mean square error
criterion = torch.nn.MSELoss()

# Train our network with a simple SGD approach
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train our network a using the entire dataset 5 times
for epoch in range(5):

    totalLoss = 0
    for i in range(len(x)):
        # Forward pass: Compute predicted y by passing x to the model
        optimizer.zero_grad()
        ypred = model(x[i])
        loss = criterion(ypred, y[i])
        totalLoss+=loss.item()
        loss.backward()
        optimizer.step()
    
    print ("Total Loss: ", totalLoss)

Total Loss:  9826.246387275416
Total Loss:  66.17708677123574
Total Loss:  26.986507382378182
Total Loss:  13.680310017999501
Total Loss:  7.697012469882144


# Step 5 - Test our Neural Network Predictions

Below we makeup some input data and see what our predictions should be vs what the neural network predicted.

In [7]:
# Data that affects the number of lemons sold in one day
weekend = 1
afternoon = 1
sunny = 1
warm = 0
sign = 1
cutedog = 0
ballons = 1
discount = 1
freshlypicked = 1
ice = 1

# Data that usually does not affect lemons sold
houseclean = 1
dogwentwalk = 1

Below we use the synthetic data generator forumla to determine what the actual result should have been.

In [8]:
numlemonssold = 0
if weekend:
    numlemonssold = afternoon + sunny*5 + warm*10 + sign*2 + cutedog + \
                    ballons+discount+freshlypicked*5 + ice*2
print ("Actual Result: ", numlemonssold)

Actual Result:  17


Below we push that same input data into the neural network and see what it predicts.

In [9]:
model.to('cpu')
x = torch.tensor([weekend, afternoon,sunny, warm, sign, cutedog, 
                 ballons, discount, freshlypicked, ice, houseclean, 
                 dogwentwalk],dtype=float)
y=model(x.float())
print("Neural Network Predicts: ", y.item())

Neural Network Predicts:  17.019847869873047


# Step 6 - Next Steps
This is just a beginning but hopefully you can see how powerful neural networks can be!  There are lots of things we would do above to make it work even better including breaking the dataset up into a training, validation and testing set and fine tune our training a lot more.  

Head over to www.leaky.ai for more tutorials and also check out our Introduction to A.I. Programming Course at https://www.leaky.ai/introduction-to-ai-programming-with-pytorch-course-beginner.

Happy Learning!

leaky.ai team